# A Deep Dive into the S&P 500: Predicting Stock Prices
Kanishk Chinnapapannagari, Aarav Naveen, Avyay Potarlanka, and Melvin Rajendran

## Introduction

In today’s evolving financial landscape, both investors and traders are constantly seeking an edge to make informed decisions. The stock market, which contains an intricate web of variables and is influenced by numerous factors, has proven to be a difficult environment to navigate.

In the past, investment-related decisions were often made based on analysis of historical trends. However, the advancement of data science and machine learning techniques has introduced a new opportunity to potentially predict future stock prices with reasonable accuracy and thus gain valuable insights.

This data science project delves into prediction of stock prices within the Standard & Poor’s 500 index, otherwise known as the S&P 500. This index contains 500 of the top companies in the United States, and it represents approximately 80% of the U.S. stock market’s total value. Hence, it serves as a strong indicator of the movement within the market. To learn more about the S&P 500 and other popular indices in the U.S., read this article: https://www.investopedia.com/insights/introduction-to-stock-market-indices/.

Throughout this project, we will follow a comprehensive data science approach that includes the following steps:
* Data collection
* Data processing
* Exploratory data analysis and data visualization
* Data analysis, hypothesis testing, and machine learning (ML)
* Insight formation

Our project aims to leverage predictive modeling techniques to provide insights to investors. The analysis herein will identify stocks that are undervalued and thus will increase in price in the near future, meaning investors should consider buying or holding shares. Likewise, it will also identify stocks that are overvalued and will soon decrease in price, indicating that investors should consider selling their position.

In [135]:
# Import necessary libraries
from bs4 import BeautifulSoup
import numpy as np
import os
import pandas as pd
import requests

## Data Collection

To gather information about the S&P 500 companies, we will be using the following dataset: https://www.kaggle.com/datasets/paultimothymooney/stock-market-data. This Kaggle dataset contains the date, volume, and prices for the NASDAQ, NYSE, and S&P 500. For the purposes of this project, we will only analyze the stock prices of companies in the S&P 500.

In [136]:
# Initialize an empty data frame to store the stock price data
price_data = pd.DataFrame()

# Initialize the path to the folder containing the data
folder_path = 'sp500-data'

# Iterate across each file in the folder by name
for file_name in os.listdir(folder_path):
    
    # Check if the current file is a CSV file
    if file_name.endswith('.csv'):
        
        # Read the current file into a temporary data frame
        temp = pd.read_csv(os.path.join(folder_path, file_name))
        
        # Extract the symbol from the current file's name
        symbol = file_name[0:-4]
        
        # Store the symbol in a new column in the temporary data frame 
        temp['Symbol'] = symbol
        
        # Concatenate the accumulating and temporary data frames
        price_data = pd.concat([price_data, temp], ignore_index = True)

# Print the first five rows of the price data frame
price_data.head()

,Date,Low,Open,Volume,High,Close,Adjusted Close,Symbol
0,16-02-1990,0.073785,0.0,940636800.0,0.079861,0.077257,0.054862,CSCO
1,20-02-1990,0.074653,0.0,151862400.0,0.079861,0.079861,0.056712,CSCO
2,21-02-1990,0.075521,0.0,70531200.0,0.078993,0.078125,0.055479,CSCO
3,22-02-1990,0.078993,0.0,45216000.0,0.081597,0.078993,0.056095,CSCO
4,23-02-1990,0.078125,0.0,44697600.0,0.079861,0.078559,0.055787,CSCO


We noticed that the Kaggle dataset does not contain sector data. For this reason, we will supplement our existing data with that which is contained in the following dataset: https://en.wikipedia.org/wiki/List_of_S%26P_500_companies. By webscraping this webpage's list of the S&P 500 companies, we can match each company in our existing data to its corresponding GICS sector and sub-industry. This will enable us to perform analysis by sector and/or sub-industry, and thus eliminate biases in our modeling.

In [137]:
# Headers for the HTTP request
headers = {
    'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/109.0.0.0 Safari/537.36',
    'From': 'pleaseletmein@gmail.com'
}

# Make an HTTP request to the Wikipedia URL and store the response
response = requests.get('https://en.wikipedia.org/wiki/List_of_S%26P_500_companies', headers = headers)

# Parse the text from the webpage as HTML
soup = BeautifulSoup(response.text, 'html.parser')

# Find the table element containing the data and both extract and store the data
table = soup.find('table')

# Read the HTML table into a data frame
sector_data = pd.read_html(str(table), flavor = 'html5lib')[0]

# Print the first five rows of the sector data frame
sector_data.head()

,Symbol,Security,GICS Sector,GICS Sub-Industry,Headquarters Location,Date added,CIK,Founded
0,MMM,3M,Industrials,Industrial Conglomerates,"Saint Paul, Minnesota",1957-03-04,66740,1902
1,AOS,A. O. Smith,Industrials,Building Products,"Milwaukee, Wisconsin",2017-07-26,91142,1916
2,ABT,Abbott,Health Care,Health Care Equipment,"North Chicago, Illinois",1957-03-04,1800,1888
3,ABBV,AbbVie,Health Care,Pharmaceuticals,"North Chicago, Illinois",2012-12-31,1551152,2013 (1888)
4,ACN,Accenture,Information Technology,IT Consulting & Other Services,"Dublin, Ireland",2011-07-06,1467373,1989


In [138]:
# Headers for the HTTP request
headers = {
    'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/109.0.0.0 Safari/537.36',
    'From': 'pleaseletmein@gmail.com'
}

# Make an HTTP request to Liberated Stock Trader's URL and store the response
response = requests.get('https://www.liberatedstocktrader.com/sp-500-companies-list-by-sector-market-cap/', headers = headers)

# Parse the text from the webpage as HTML
soup = BeautifulSoup(response.text, 'html.parser')

# Find the table element containing the data and both extract and store the data
table = soup.findAll('table')[1]

# Read the HTML table into a data frame
mcap = pd.read_html(str(table), flavor = 'html5lib')[0]

# Print the first five rows of the sector data frame
mcap.head()

,0,1,2,3,4
0,Ticker,Company,Sector,Market Capitalization,PE Ratio
1,MRO,Marathon Oil Corporation,Energy Minerals,17388156106,2262.602
2,MO,"Altria Group, Inc.",Consumer Non-Durables,81364639112,522.1622
3,ANET,"Arista Networks, Inc.",Electronic Technology,35807001092,471.4286
4,MTB,M&T Bank Corporation,Finance,25831948062,397.9184


## Data Processing

In [139]:
# Clean sector data

# Rename columns
sector_data = sector_data.rename(columns={'GICS Sector': 'Sector', 'GICS Sub-Industry': 'Industry'})

# Drop unneeded columns
sector_data = sector_data.drop('CIK', axis = 1)
sector_data = sector_data.drop('Date added', axis = 1)
sector_data = sector_data.drop('Founded', axis = 1)
sector_data = sector_data.drop('Headquarters Location', axis = 1)

# Print first 5 rows
sector_data.head()

,Symbol,Security,Sector,Industry
0,MMM,3M,Industrials,Industrial Conglomerates
1,AOS,A. O. Smith,Industrials,Building Products
2,ABT,Abbott,Health Care,Health Care Equipment
3,ABBV,AbbVie,Health Care,Pharmaceuticals
4,ACN,Accenture,Information Technology,IT Consulting & Other Services


In [140]:
# Clean market cap data

# Name the data frame's columns
mcap.columns = ['Symbol', 'Company', 'Sector', 'Market Cap', 'PE Ratio']

# Drop the first row of the data frame because it contains the feature names
mcap = mcap.drop(0)

# Drop everything but ticker and market cap
mcap = mcap.drop('Company', axis = 1)
mcap = mcap.drop('Sector', axis = 1)
mcap = mcap.drop('PE Ratio', axis = 1)

# Print the first five rows of the sector data frame
mcap.head()

,Symbol,Market Cap
1,MRO,17388156106
2,MO,81364639112
3,ANET,35807001092
4,MTB,25831948062
5,CI,94846357195


In [141]:
# Merge into sector data so we can sort by market cap
sector_data = pd.merge(mcap, sector_data, on = 'Symbol')

# Show first 5 rows
sector_data.head()

,Symbol,Market Cap,Security,Sector,Industry
0,MRO,17388156106,Marathon Oil,Energy,Oil & Gas Exploration & Production
1,MO,81364639112,Altria,Consumer Staples,Tobacco
2,ANET,35807001092,Arista Networks,Information Technology,Communications Equipment
3,MTB,25831948062,M&T Bank,Financials,Regional Banks
4,CI,94846357195,Cigna,Health Care,Managed Health Care


In [142]:
# Store only top 5 by market cap in each sector
topsect_df = pd.DataFrame()

# Get unique sectors in an array
sectors = sector_data['Sector'].unique()

# Traverse unique sectors
for sector in sectors:
    # Create a sub dataframe for the sector
    sub_df = sector_data[sector_data['Sector'] == sector]

    # Sort sub dataframe by market cap
    sub_df = sub_df.sort_values('Market Cap', ascending=False, key=lambda x: x.astype(int))

    # Merge into dataframe of all sectors' top 5
    topsect_df = pd.concat([topsect_df, sub_df.head(5)], ignore_index = True)

# Print 5 rows
topsect_df.head()

,Symbol,Market Cap,Security,Sector,Industry
0,XOM,466273190232,ExxonMobil,Energy,Integrated Oil & Gas
1,CVX,342408717940,Chevron Corporation,Energy,Integrated Oil & Gas
2,COP,149727915989,ConocoPhillips,Energy,Oil & Gas Exploration & Production
3,SLB,82243638916,Schlumberger,Energy,Oil & Gas Equipment & Services
4,EOG,76624961922,EOG Resources,Energy,Oil & Gas Exploration & Production


In [144]:
# Merge both data frames into a single data frame
data = pd.merge(price_data, sector_data, on = 'Symbol')

# Print the first five rows of the data frame
data.head()

,Date,Low,Open,Volume,High,Close,Adjusted Close,Symbol,Market Cap,Security,Sector,Industry
0,16-02-1990,0.073785,0.0,940636800.0,0.079861,0.077257,0.054862,CSCO,201297048340,Cisco,Information Technology,Communications Equipment
1,20-02-1990,0.074653,0.0,151862400.0,0.079861,0.079861,0.056712,CSCO,201297048340,Cisco,Information Technology,Communications Equipment
2,21-02-1990,0.075521,0.0,70531200.0,0.078993,0.078125,0.055479,CSCO,201297048340,Cisco,Information Technology,Communications Equipment
3,22-02-1990,0.078993,0.0,45216000.0,0.081597,0.078993,0.056095,CSCO,201297048340,Cisco,Information Technology,Communications Equipment
4,23-02-1990,0.078125,0.0,44697600.0,0.079861,0.078559,0.055787,CSCO,201297048340,Cisco,Information Technology,Communications Equipment


## Exploratory Data Analysis and Data Visualization

## Data Analysis, Hypothesis Testing, and Machine Learning

## Insights